# Atualizar:
- Melhorar função de testar similaridade
- Adicionar memória com limite de mensagens no chat
- Melhorar interface Gradio

In [1]:
import gradio as gr
import chromadb
#import asyncio
import nest_asyncio
from glob import glob
from PyPDF2 import PdfReader
from os import path, getenv
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core.schema import TextNode
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
#import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from typing import List, Dict


# Implementação Chroma + LLamaIndex

In [2]:
class Config:
    def __init__(self):
        load_dotenv()
        #self.llm_model = getenv('llm_model')
        self.llm_model = "mistralai/Mistral-Nemo-Instruct-2407"
        self.embedding_model = getenv('embedding_model')
        self.api_key = getenv('api_key')
        self.db_path = getenv('db_path')
        self.input_dir = getenv('input_dir')
        self.collection_name = getenv('collection_name')
        self.parser_key = getenv('parser_key')
        self.connection = None

    def connect_db(self):
        try:
            db_client = chromadb.PersistentClient(path=self.db_path)
            collection = db_client.get_or_create_collection(self.collection_name)
            self.connection = ChromaVectorStore(chroma_collection=collection)
            
            print("Conexão ao ChromaDB estabelecida com sucesso!")
            return self.connection
        except Exception as e:
            raise Exception(f"Erro ao conectar ao banco de dados: {str(e)}")
        

## Uso do LlamaParse + SemanticSplitterNodeParser

In [3]:

class GenarateIndex:
    def __init__(self):
        self.config = Config()
        self.vector_store = self.config.connect_db()
        self.llm_model = self.config.llm_model
        self.embed_model_name = self.config.embedding_model
        self.input_dir = self.config.input_dir
        self.api_key = self.config.api_key
        self.parser_key = self.config.parser_key
        self.nodes = None

    def generate_index(self):
        try:
            self.embedding_model = HuggingFaceEmbedding(model_name=self.embed_model_name)
            Settings.node_parser = SemanticSplitterNodeParser(buffer_size=1,
                                                              include_metadata = True, 
                                                              breakpoint_percentile_threshold=50,
                                                              include_prev_next_rel = True,
                                                              embed_model=self.embedding_model)
            
            self.nodes = Settings.node_parser.get_nodes_from_documents(self.documents)
            self.index = VectorStoreIndex(
                nodes = self.nodes,
                vector_store=self.vector_store,
                embed_model=self.embedding_model,
                show_progress=True,
            )

            print("Índice gerado com sucesso!")
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao gerar índice: {str(e)}")

    async def process_pdfs(self, input_dir):
        nest_asyncio.apply()
        """
        Processa arquivos PDF em um diretório e extrai texto.
        """
        files = glob(f"{input_dir}/*.pdf")

        try:
            parser = LlamaParse(api_key=self.parser_key, result_type="markdown",)

            for file in files:
               
               file_extractor = {".pdf": parser}
               reader = SimpleDirectoryReader(
                   input_files=[file],
                   file_extractor=file_extractor
               )
               
               # Carregar dados de forma assíncrona
               self.documents = await reader.aload_data()
               
               if not self.documents:
                 raise ValueError(f"O PDF {file} parece não conter texto processável.")
            
            return self.documents
        except Exception as e:
            raise Exception(f"Erro ao processar PDFs: {str(e)}")
        
    async def execute(self):
        try:
            nest_asyncio.apply()
            await self.process_pdfs(self.input_dir)
            self.generate_index()
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao executar o processo: {str(e)}")
        



## Uso do llamaparse + tratamento em função de chunks por qtd de palavras

In [ ]:

        

class GenarateIndex:
    def __init__(self):
        self.config = Config()
        self.vector_store = self.config.connect_db()
        self.llm_model = self.config.llm_model
        self.embed_model_name = self.config.embedding_model
        self.input_dir = self.config.input_dir
        self.api_key = self.config.api_key
        self.parser_key = self.config.parser_key
        self.nodes = None

    def generate_index(self):
        try:
            Settings.embed_model = HuggingFaceEmbedding(model_name=self.embed_model_name)
            Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=50)

            self.index = VectorStoreIndex(
                nodes = self.nodes,
                vector_store=self.vector_store,
                embed_model=self.embedding_model,
                show_progress=True,
            )

            print("Índice gerado com sucesso!")
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao gerar índice: {str(e)}")

    def split_into_chunks(self, text, chunk_size=150, overlap=50):
        """
        Divide o texto em chunks de palavras com overlap especificado.

        :param text: Texto completo que será dividido.
        :param chunk_size: Número de palavras em cada chunk.
        :param overlap: Quantidade de palavras que se sobrepõem entre chunks.
        :return: Lista de chunks de texto.
        """
        words = text.split()  # Divide o texto em uma lista de palavras
        chunks = []
        start = 0
        while start < len(words):
            end = start + chunk_size
            chunk = " ".join(words[start:end])  # Junta as palavras para formar o chunk
            chunks.append(chunk)
            start += chunk_size - overlap  # Avança, considerando o overlap
        return chunks

    async def process_pdfs(self, input_dir):
        nest_asyncio.apply()
        """
        Processa arquivos PDF em um diretório, extrai texto e o divide em chunks.
        """
        files = glob(f"{input_dir}/*.pdf")
        text_nodes = []

        try:
            parser = LlamaParse(api_key=self.parser_key, result_type="markdown",)

             #Processar cada arquivo PDF
            for file in files:
                # Usar SimpleDirectoryReader com LlamaParse para extração de texto
                file_extractor = {".pdf": parser}
                reader = SimpleDirectoryReader(
                    input_files=[file],
                    file_extractor=file_extractor
                )
                
                # Carregar dados de forma assíncrona
                documents = await reader.aload_data()
                if not documents:
                    raise ValueError(f"O PDF {file} parece não conter texto processável.")
            
                # Concatenar texto dos documentos e dividir em chunks
                text = " ".join([doc.text for doc in documents])
                chunks = self.split_into_chunks(text)
            
                # Criar nós de texto para o índice
                for idx, chunk in enumerate(chunks, start=1):
                    text_nodes.append(TextNode(
                        text=chunk,
                        metadata={"filename": file, "num_chunk": idx}
                    ))
            
                print(f"Texto extraído do arquivo {file} com sucesso!")
            
            self.nodes = list(text_nodes)

            if not self.nodes:
                 raise ValueError(f"O PDF {file} parece não conter texto processável.")
            
            return self.nodes
        except Exception as e:
            raise Exception(f"Erro ao processar PDFs: {str(e)}")
        
    async def execute(self):
        try:
            nest_asyncio.apply()
            await self.process_pdfs(self.input_dir)
            self.generate_index()
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao executar o processo: {str(e)}")
        



## Pdfreader

In [14]:
class GenarateIndex:
    def __init__(self):
        self.config = Config()
        self.vector_store = self.config.connect_db()
        self.llm_model = self.config.llm_model
        self.embed_model_name = self.config.embedding_model
        self.input_dir = self.config.input_dir
        self.api_key = self.config.api_key
        self.nodes = None

    def generate_index(self):
        try:
            #self.embedding_model = HuggingFaceEmbedding(model_name=self.embed_model_name)
            Settings.embed_model = HuggingFaceEmbedding(model_name=self.embed_model_name, trust_remote_code=True)
            Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=50)
            Settings.num_output = 512
            Settings.context_window = 3900
            self.index = VectorStoreIndex(
                nodes = self.nodes,
                vector_store=self.vector_store,
                #embed_model=self.embedding_model,
                show_progress=True,
            )
            print("Índice gerado com sucesso!")
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao gerar índice: {str(e)}")

    def split_into_chunks(self, text, chunk_size=150, overlap=50):
        """
        Divide o texto em chunks de palavras com overlap especificado.

        :param text: Texto completo que será dividido.
        :param chunk_size: Número de palavras em cada chunk.
        :param overlap: Quantidade de palavras que se sobrepõem entre chunks.
        :return: Lista de chunks de texto.
        """
        words = text.split()  # Divide o texto em uma lista de palavras
        chunks = []
        start = 0
        while start < len(words):
            end = start + chunk_size
            chunk = " ".join(words[start:end])  # Junta as palavras para formar o chunk
            chunks.append(chunk)
            start += chunk_size - overlap  # Avança, considerando o overlap
        return chunks
    
    def process_pdfs(self, input_dir):
        files = glob(f"{input_dir}/*.pdf")
        text_nodes = []
        try:
            for file in files:
                reader = PdfReader(file)
                text = ""
                
                for page in reader.pages:
                    extracted_text = page.extract_text().replace('\n', ' ').replace('  ', ' ')
                    print
                    if extracted_text:
                        text += extracted_text + " "
                
                if not text.strip():
                    raise ValueError(f"O PDF {file} parece não conter texto processável.")
                
                chunks = self.split_into_chunks(text)
                
                # Armazenando os chunks de texto e metadados
                for chunk in chunks:
                    cont =+ 1
                    text_nodes.append(TextNode(
                    text=chunk,
                    metadata={"filename": file, "num_chunk": cont},
                    
                ))
                # Armazenando o texto extraído e metadados
                
                print(f"Texto extraído do arquivo {file} com sucesso!")
            self.nodes = list(text_nodes)
            return self.nodes
        except Exception as e:
            raise Exception(f"Erro ao transformar texto: {str(e)}")
        
    def execute(self):
        try:
            self.process_pdfs(self.input_dir)
            self.generate_index()
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao executar o processo: {str(e)}")
        



# Chatbot

In [4]:
class ChatBot:
    def __init__(self, index):
        self.config = Config()
        self.api_key = self.config.api_key
        self.llm_model = self.config.llm_model
        self.index = index
        

    def initialize_llm(self):
        try:
            self.prompt = (
                    "Essas são suas diretrizes para interagir com os usuários: "
                    "Seu nome é BOOT. "
                    "Você é um assistente virtual projetado para fornecer informações sobre diversos tópicos. "
                    "Responda exclusivamente em !![]português. "
                    "Caso o usuário inicie a conversa com uma saudação, cumprimento ou pergunte quem você é ou sobre voê, APENAS apresente-se de forma cordial e simpática. "
                    "Responda com base no que sabe e adapte suas respostas de forma a se encaixar perfeitamente na pergunta ou necessidade do usuário. "
                    "!![]Não use os termos 'Resposta:' ou 'Pergunta:' ao iniciar suas mensagens. "
                    "Seja educado em todas as interações."
                    "Sempre atenda aos pedidos do usuário em !![]português."
                    )
            Settings.llm = HuggingFaceInferenceAPI(model_name=self.llm_model, 
                                                   token=self.api_key, 
                                                   system_prompt=self.prompt,
                                                   generate_kwargs={"temperature": 0.9})

            Settings.num_output = 1000
            Settings.context_window = 5000

            chat_engine = self.index.as_chat_engine(
                chat_mode="condense_question",
                verbose=False
            )

            return chat_engine
            
        except Exception as e:
            raise Exception(f"Erro ao inicializar Modelo: {str(e)}")
        
    def get_similarity(self, query, context):

        """Calcula a similaridade entre a pergunta e o contexto usando TF-IDF e cosine similarity."""
        filtered_context = []
        for node in context.split('|'):

            vectorizer = TfidfVectorizer().fit_transform([query, node])
            similarity_matrix = cosine_similarity(vectorizer[0:1], vectorizer[1:])
      
            
            if similarity_matrix[0][0] > 0.01:
                print(node)
                filtered_context.append(node)
                print(f"Similaridade: {similarity_matrix}")
                print("---------------------------")

        filtered_context = "\n".join(filtered_context)
        return filtered_context
        
    def search(self, query):

        try:

            query_engine = self.index.as_query_engine(
                llm=HuggingFaceInferenceAPI(model_name=self.llm_model, token=self.api_key),
                response_mode="tree_summarize",
                similarity_top_k=5,
                verbose=False
            )
            response = query_engine.query(query)
            context = "|".join([doc.node.text for doc in response.source_nodes])

            filtered_context = self.get_similarity(query, context)
            print(f"Contexto: {filtered_context}")

            return response, filtered_context

        except Exception as e:
            raise Exception(f"Erro ao realizar a busca: {str(e)}")
        
        
    def respond(self, message, history=None):
        """Gera uma resposta baseada no contexto da pesquisa."""
        try:
            
            # Busca o contexto no ChromaDB
            chat_engine = self.initialize_llm()
            response, context = self.search(message)

            message_context = f"Contexto: {context} || Pergunta: {message}"

            # Gera a resposta usando o motor de chat
            response = chat_engine.chat(message_context)
            print(message_context)
            return response.response
        
        except Exception as e:
            return f"Erro ao processar mensagem: {str(e)}"

    def interface(self): 
        with gr.Blocks(theme=gr.themes.Base()) as demo:
            # CSS para personalizar o tamanho da área do chatbot
            css = """
            .gr-chatbot {
                height: 600px !important; /* Ajuste a altura desejada */
                overflow-y: auto; /* Permite rolagem se necessário */
            }
            .toast-wrap { 
                display: none !important; 
            }
            """
            
            gr.ChatInterface(
                self.respond, 
                type="messages",
                title="Boot",  
                css=css, 
                examples=["hello", "hola", "merhaba", "oi"],
            ) 
        
        
        return demo.queue().launch(debug=True)

In [5]:
generate_index = GenarateIndex()
index = await generate_index.execute()

Conexão ao ChromaDB estabelecida com sucesso!
Started parsing the file under job_id 3011dd08-82f8-49e0-9628-b0650da0240f


Generating embeddings:   0%|          | 0/154 [00:00<?, ?it/s]

Índice gerado com sucesso!


In [6]:
chatbot = ChatBot(index)
chatbot.respond("Me resuma os resultados da magalu no útimo trimestre")

O Magalu não teria a escala, a abrangência e resultados mais resilientes sem isso.

As mais de duas dezenas de operações adquiridas, além de outras desenvolvidas do zero como a Magalu Cloud, foram integradas e conectadas e parte delas se transformou em quatro pilares de geração de resultado: o Magalu Bank, em serviços financeiros, o Magalu Ads, em retail media e conteúdo, o Magalu Cloud, em Maas (tecnologia), e o Magalog, em logística. Netshoes, Época, Zattini, KaBum!, Aiqfome e Estante Virtual hoje estão 100% conectadas ao ecossistema, contribuem para seu fortalecimento ao mesmo tempo em que se beneficiam dele. 
Similaridade: [[0.0869909]]
---------------------------
Ao longo do trimestre finalizamos também o aumento de capital na Luizacred no montante de R$300 milhões e recompramos cerca de R$100 milhões de dívida (referentes à 10ª emissão de debêntures). Com isso, o caixa líquido do Magalu ao término do trimestre alcançou R$1,8 bilhão.

Tem Ali no Magalu: A parceria entre Magalu e A

' No último trimestre, a Magalu apresentou um EBITDA de 2,9 bilhões de reais, com um crescimento de 40% em relação ao ano anterior. O lucro líquido foi de 239 milhões de reais, sendo o quarto balanço consecutivo de resultado positivo. A geração de caixa operacional foi de 571 milhões de reais. A posição de caixa total do Magalu em setembro foi de 6,6 bilhões de reais, com um aumento de 162 milhões no trimestre. A empresa também realizou uma recompra de cerca de 100 milhões de reais em debêntures.'

In [ ]:
chatbot.interface()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Essa parceria permitirá ao Magalu ampliar sua participação de mercado em categorias de tickets baixos por meio do marketplace e também alavancar as vendas do 1P para uma nova base de clientes.

O Fulfillment já é responsável por 24% dos pedidos do marketplace, um significativo aumento de 10 p.p. 
Similaridade: [[0.18036443]]
---------------------------
O e-commerce atingiu R$11 bilhões em vendas no período. As vendas no e-commerce com estoque próprio totalizaram R$6,5 bilhões, e o DIFAL totalmente repassado contribuiu para a expansão da margem bruta. Já as vendas do marketplace alcançaram R$4,5 bilhões no mesmo período e representaram 41% das vendas online.

A margem bruta atingiu 31,5%, crescendo de 1,1 p.p. 
Similaridade: [[0.18429267]]
---------------------------
# Crescimento das Vendas Totais (em %)


Similaridade: [[0.07357475]]
---------------------------
Crescimento das Vendas Mesmas Lojas Físicas
Similaridade: [[0.06655925]]
---------------------------
Crescimento do E-commerc

# Teste llama-parser

In [27]:
# bring in dependencies
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from os import getenv
from dotenv import load_dotenv
import asyncio
import nest_asyncio

# Habilitar nested asyncio (necessário em Jupyter ou ambientes interativos)
nest_asyncio.apply()

# Configurar o parser
load_dotenv()
parser_key = getenv('parser_key')
parser = LlamaParse(
    api_key=parser_key,
    result_type="markdown"  # ou "text"
)

# Função assíncrona para carregar os documentos
async def load_documents():
    # Configurar o extrator de arquivos
    file_extractor = {".pdf": parser}
    # Usar SimpleDirectoryReader para carregar o arquivo
    reader = SimpleDirectoryReader(
        input_files=['data/TESTE_CONTO.pdf'],
        file_extractor=file_extractor
    )
    # Carregar dados (assíncrono)
    documents = await reader.aload_data()
    return documents

# Executar no loop de eventos
if __name__ == "__main__":
    # Rodar a função assíncrona
    documents = asyncio.run(load_documents())
    print(documents)


Started parsing the file under job_id 6dae1a05-e595-4fbc-93bc-14180669915b
[Document(id_='f89502c5-98ab-4be7-8370-6c433aaae492', embedding=None, metadata={'file_path': 'data\\TESTE_CONTO.pdf', 'file_name': 'TESTE_CONTO.pdf', 'file_type': 'application/pdf', 'file_size': 69984, 'creation_date': '2025-01-03', 'last_modified_date': '2025-01-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# A raposa e o tucano\n\nCerta vez uma raposa convidou o tucano para o jantar. A comida era um mingau servido em cima de uma pedra. O pobre tucano teve dificuldade para comer e machucou seu longo bico.\n\nCom raiva, o tucano quis se vingar. Assim, convid

In [73]:
import re
from typing import List, Dict
from llama_index.core import SimpleDirectoryReader
from llama_index.core.schema import TextNode
from llama_parse import LlamaParse
import nest_asyncio

class MarkdownPreprocessor:
    def __init__(self, parser_key: str):
        self.parser_key = parser_key
        self.nodes: List[TextNode] = []
    
    def extract_section_metadata(self, text: str) -> List[Dict]:
        """
        Extrai metadados de seções e divide o texto em parágrafos.
        Retorna uma lista de dicionários com informações das seções e seus parágrafos.
        """
        sections = []
        current_headers = {1: None, 2: None, 3: None, 4: None, 5: None, 6: None}
        
        # Regex para identificar headers markdown (# Header)
        header_pattern = re.compile(r'^(#{1,6})\s+(.+)$', re.MULTILINE)
        
        # Encontrar todas as posições de headers
        headers = list(header_pattern.finditer(text))
        
        if not headers:
            # Se não houver headers, trata o texto inteiro como uma seção
            paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
            for idx, paragraph in enumerate(paragraphs, 1):
                sections.append({
                    "text": paragraph,
                    "metadata": {
                        "paragraph_number": idx,
                        "total_paragraphs": len(paragraphs)
                    }
                })
            return sections
        
        for i, match in enumerate(headers):
            level = len(match.group(1))  # Número de #
            title = match.group(2).strip()
            start = match.start()
            
            # Define o fim da seção atual
            end = headers[i + 1].start() if i < len(headers) - 1 else len(text)
            
            # Atualiza headers ativos para este nível
            current_headers[level] = title
            # Limpa subníveis
            for l in range(level + 1, 7):
                current_headers[l] = None
            
            # Cria hierarquia completa
            hierarchy = {
                f"h{l}": current_headers[l]
                for l in range(1, 7)
                if current_headers[l] is not None
            }
            
            # Extrai o texto da seção atual
            section_text = text[start:end].strip()
            
            # Remove o header do início do texto da seção
            section_text = re.sub(r'^#{1,6}\s+.+\n', '', section_text, flags=re.MULTILINE)
            
            # Divide em parágrafos
            paragraphs = [p.strip() for p in section_text.split('\n\n') if p.strip()]
            
            # Cria um node para cada parágrafo
            for p_idx, paragraph in enumerate(paragraphs, 1):
                sections.append({
                    "text": paragraph,
                    "metadata": {
                        "header_level": level,
                        "header_title": title,
                        "paragraph_number": p_idx,
                        "total_paragraphs": len(paragraphs),
                        **hierarchy
                    }
                })
        
        return sections

    async def process_files(self, files: List[str]) -> List[TextNode]:
        """
        Processa arquivos PDF e gera TextNodes com metadados baseados na estrutura markdown,
        dividindo o conteúdo em parágrafos.
        """
        nest_asyncio.apply()
        
        text_nodes = []
        
        parser = LlamaParse(
            api_key=self.parser_key,
            result_type="markdown",
        )

        for file in files:
            try:
                # Configurar extrator
                file_extractor = {".pdf": parser}
                reader = SimpleDirectoryReader(
                    input_files=[file],
                    file_extractor=file_extractor
                )
                
                # Carregar documentos
                documents = await reader.aload_data()
                if not documents:
                    raise ValueError(f"O PDF {file} parece não conter texto processável.")
                
                # Processar cada documento
                for doc in documents:
                    # Extrair seções e parágrafos com metadados
                    sections = self.extract_section_metadata(doc.text)
                    
                    # Criar TextNodes para cada parágrafo
                    for idx, section in enumerate(sections, start=1):
                        metadata = {
                            "filename": file,
                            "chunk_number": idx,
                            **section["metadata"]
                        }
                        
                        text_nodes.append(TextNode(
                            text=section["text"],
                            metadata=metadata
                        ))
                
                print(f"Texto extraído do arquivo {file} com sucesso!")
            
            except Exception as e:
                print(f"Erro ao processar {file}: {str(e)}")
                continue
        
        self.nodes = text_nodes
        return text_nodes

    def get_nodes(self) -> List[TextNode]:
        """Retorna os nodes processados."""
        return self.nodes

In [74]:

mark = MarkdownPreprocessor(parser_key)
teste = asyncio.run(mark.process_files(['data/TESTE_CONTO.pdf']))
teste

Started parsing the file under job_id e381d3b7-52f0-4b73-8f32-7b42db13191e
Texto extraído do arquivo data/TESTE_CONTO.pdf com sucesso!


[TextNode(id_='1217761a-6f8e-4c3c-8f6c-9be655bbf886', embedding=None, metadata={'filename': 'data/TESTE_CONTO.pdf', 'chunk_number': 1, 'header_level': 1, 'header_title': 'A raposa e o tucano', 'paragraph_number': 1, 'total_paragraphs': 7, 'h1': 'A raposa e o tucano'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Certa vez uma raposa convidou o tucano para o jantar. A comida era um mingau servido em cima de uma pedra. O pobre tucano teve dificuldade para comer e machucou seu longo bico.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='f38db1f8-914d-4376-ae86-ced9cb0d3cb0', embedding=None, metadata={'filename': 'data/TESTE_CONTO.pdf', 'chunk_number': 2, 'header_level': 1, 'header_title': 'A raposa e o tucano', 'paragraph_number': 2, 'total_paragraphs': 7, 'h1': 'A raposa e o tucano'}, 